# Tulio Quijada - Coursera - Capstone 

## Import libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner
import requests
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


## Using BeautifulSoup library

In [90]:
from bs4 import BeautifulSoup
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')


## Seting up - Data Set

In [96]:
#creating data set from return of BeautifulSoup library
My_table = soup.find_all('table')[0]
df_table = pd.read_html(str(My_table))[0]

#Removing 'Not assigned' values from series 'Borough'
df_table.drop(df_table[df_table.Borough == 'Not assigned'].index, inplace=True)
df_table = df_table.reset_index (drop=True)

#Removing 'Not assigned' values from series 'Neighbourhood' and seting values = to 'Borough' values
df_table.loc[df_table['Neighbourhood'] == 'Not assigned', ['Neighbourhood'] ] =   df_table['Borough']

#Group by 'Postcode', 'Borough' and seting list for common Neighbourhood
df_table_f = df_table.groupby(('Postcode', 'Borough'))
result = df_table_f['Neighbourhood'].unique()
df_toronto = pd.DataFrame(result)
df_toronto.reset_index( inplace=True)

#seting Neighbourhood
df_toronto['Expand'] = df_toronto.Neighbourhood.apply(', '.join)
del df_toronto['Neighbourhood']
df_toronto = df_toronto.rename(columns={'Expand' : 'Neighbourhood'})

#df_toronto Shape
df_toronto.shape()

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  del sys.path[0]


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Seting up - reading csv file - Toronto Geo Coordinates

In [105]:
toronto_geo_cordi = pd.read_csv('Geospatial_Coordinates.csv')
toronto_geo_cordi = toronto_geo_cordi.rename(columns={'Postal Code' : 'Postcode'})
toronto_geo_cordi.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge df_toronto with toronto_geo_cordi

In [107]:
total_df = pd.merge(df_toronto, toronto_geo_cordi, on='Postcode')
total_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
